In [ ]:
# installs

# notebook/colab
! pip install scikit-plot

In [20]:
# imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots

# what we need for today
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics 

import scikitplot as skplt

# color maps
from matplotlib import cm

# for distance and h-clustering
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist, squareform


# sklearn does have some functionality too, but mostly a wrapper to scipy
from sklearn.metrics import pairwise_distances 
from sklearn.preprocessing import StandardScaler

## Loading the Data

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
path = "/content/stock-fundamentals.csv"
df = pd.read_csv(path)

In [84]:
df.head(3)

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker,quarter_end
0,NaN,3588919000,3588919000,1,118318000000,5.693800e+10,87263000000,2.031500e+10,30572000000,483000000,0,49881000000,51561000000,9.562000e+09,2333000000,2333000000,0.63,0.61,0.19,5.460000e+08,1.920000e+09,-1.002300e+10,-7631000000,10824000000,421000000,49.23,52.11,46.34,0.0982,0.0305,8.52,4.90,54.10,4.37,0.7371,1.6865,0.2584,0.1001,0.30,0.03,2.8028,ORCL,2018-11-30
1,NaN,261043625,261043625,1,53289000000,1.343800e+10,33995000000,9.496000e+09,19294000000,0,0,6908000000,16399000000,1.782400e+10,935000000,933000000,3.56,3.51,0.65,1.478000e+09,-1.480000e+09,-2.300000e+08,-246000000,2123000000,1455000000,233.57,259.25,207.90,0.2586,0.0950,73.91,3.21,13.20,12.17,0.1301,0.8500,0.3621,0.0722,1.31,0.09,1.4151,FDX,2018-11-30
2,NaN,359091176,359091176,1,7002400000,5.290000e+09,4577900000,4.306800e+09,2424500000,0,0,938600000,0,8.589000e+08,235800000,235800000,0.66,0.65,0.56,2.235000e+08,1.203000e+08,-2.733000e+08,70500000,510600000,36600000,69.45,76.02,62.89,0.4361,0.1366,6.75,10.44,26.51,21.27,0.8085,0.0000,0.3462,0.2799,0.49,0.52,1.2283,PAYX,2018-11-30


In [85]:
df.shape

(755, 43)

In [86]:
df.describe()

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio
count,0.0,7.550000e+02,7.550000e+02,755.0,7.550000e+02,6.330000e+02,7.550000e+02,6.330000e+02,7.550000e+02,7.550000e+02,7.550000e+02,7.550000e+02,7.550000e+02,7.520000e+02,7.550000e+02,7.550000e+02,755.000000,755.000000,755.000000,7.530000e+02,7.530000e+02,7.530000e+02,7.550000e+02,7.550000e+02,7.550000e+02,712.000000,712.000000,712.000000,692.000000,754.000000,755.000000,671.000000,616.000000,755.000000,653.000000,712.000000,712.000000,652.000000,748.000000,753.000000,633.000000
mean,NaN,4.911493e+08,4.911493e+08,1.0,6.107129e+10,7.512716e+09,4.941542e+10,5.626906e+09,1.135538e+10,3.004828e+08,2.471578e+08,6.943087e+09,1.325604e+10,4.567319e+09,4.325207e+08,4.101034e+08,15.986159,15.971364,0.308014,8.629677e+08,-4.121679e+08,-4.313689e+08,1.039072e+07,3.763882e+09,2.717754e+08,525.730843,571.091517,480.370028,0.151804,0.058858,328.093404,5.691684,39.212354,18.475483,0.591236,1.240045,0.372819,0.137426,0.766738,6.622138,1.838612
std,NaN,8.853124e+08,8.853124e+08,0.0,2.343986e+11,1.487448e+10,2.185426e+11,1.137258e+10,2.763748e+10,9.889860e+08,1.832846e+09,1.714686e+10,1.234353e+11,9.220713e+09,1.828721e+09,1.847186e+09,410.489825,410.490229,0.366153,2.350119e+09,2.514899e+09,2.735644e+09,2.135659e+09,2.024107e+10,6.412763e+08,11543.079379,12535.221949,10550.938854,0.197467,0.078665,8325.804594,12.727791,78.579622,37.210077,1.576597,3.693016,0.192265,0.118882,0.676155,149.894366,1.378521
min,NaN,1.641681e+06,1.641681e+06,1.0,1.414119e+07,1.324407e+07,1.272664e+06,1.250634e+06,-1.172000e+10,-2.647100e+08,0.000000e+00,0.000000e+00,0.000000e+00,3.414002e+06,-2.369800e+10,-2.388900e+10,-15.520000,-15.520000,0.000000,-7.261000e+09,-3.346100e+10,-2.763500e+10,-2.384200e+10,0.000000e+00,-6.820000e+08,1.060000,1.770000,0.120000,-0.939000,-0.446700,-65.800000,0.170000,1.440000,0.000000,0.000000,0.000000,0.002700,0.000600,0.010000,-18.740000,0.192100
25%,NaN,1.265114e+08,1.265114e+08,1.0,5.938500e+09,1.524000e+09,3.330460e+09,8.977000e+08,1.736500e+09,0.000000e+00,0.000000e+00,2.708150e+08,9.748150e+08,8.339000e+08,6.028600e+07,5.950000e+07,0.295000,0.290000,0.000000,9.747500e+07,-4.190000e+08,-3.730000e+08,-8.950000e+07,2.541105e+08,2.305300e+07,34.250000,37.985000,31.002500,0.069100,0.016350,9.290000,1.805000,14.737500,2.275000,0.084200,0.278800,0.229850,0.058275,0.320000,0.120000,1.047500
50%,NaN,2.442958e+08,2.442958e+08,1.0,1.383924e+10,3.014427e+09,8.873000e+09,2.137486e+09,4.554000e+09,5.000000e+05,0.000000e+00,1.894898e+09,3.245500e+09,1.743260e+09,1.693810e+08,1.662000e+08,0.800000,0.790000,0.205000,2.890000e+08,-9.500000e+07,-1.060000e+08,1.586000e+06,6.957180e+08,7.120000e+07,65.430000,70.750000,60.005000,0.137200,0.050600,18.320000,2.960000,21.790000,10.080000,0.302900,0.636700,0.362650,0.103850,0.600000,0.770000,1.498800
75%,NaN,4.788721e+08,4.788721e+08,1.0,3.648200e+10,6.632000e+09,2.493320e+10,5.031000e+09,1.044722e+10,1.200685e+08,0.000000e+00,6.303200e+09,8.684768e+09,4.063750e+09,4.925000e+08,4.890000e+08,1.550000,1.535000,0.462500,7.240220e+08,-1.340000e+07,-1.034000e+06,1.090000e+08,1.925078e+09,2.239500e+08,111.145000,119.515000,102.580000,0.233250,0.090200,32.510000,5.475000,35.192500,24.685000,0.579900,1.193225,0.504950,0.179975,0.962500,1.650000,2.127800
max,NaN,9.814197e+09,9.814197e+09,1.0,3.401

In [87]:
df.dtypes

Quarter end                                   float64
Shares                                          int64
Shares split adjusted                           int64
Split factor                                    int64
Assets                                          int64
Current Assets                                float64
Liabilities                                     int64
Current Liabilities                           float64
Shareholders equity                             int64
Non-controlling interest                        int64
Preferred equity                                int64
Goodwill & intangibles                          int64
Long-term debt                                  int64
Revenue                                       float64
Earnings                                        int64
Earnings available for common stockholders      int64
EPS basic                                     float64
EPS diluted                                   float64
Dividend per share          

In [88]:
df[~(df['Current Assets'].isnull()) & (df['Current Liabilities'].isnull())]

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker,quarter_end


# Data Cleaning 

In [89]:
df.index = df['ticker']

In [90]:
df.head(3)

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker,quarter_end
ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ORCL,NaN,3588919000,3588919000,1,118318000000,5.693800e+10,87263000000,2.031500e+10,30572000000,483000000,0,49881000000,51561000000,9.562000e+09,2333000000,2333000000,0.63,0.61,0.19,5.460000e+08,1.920000e+09,-1.002300e+10,-7631000000,10824000000,421000000,49.23,52.11,46.34,0.0982,0.0305,8.52,4.90,54.10,4.37,0.7371,1.6865,0.2584,0.1001,0.30,0.03,2.8028,ORCL,2018-11-30
FDX,NaN,261043625,261043625,1,53289000000,1.343800e+10,33995000000,9.496000e+09,19294000000,0,0,6908000000,16399000000,1.782400e+10,935000000,933000000,3.56,3.51,0.65,1.478000e+09,-1.480000e+09,-2.300000e+08,-246000000,2123000000,1455000000,233.57,259.25,207.90,0.2586,0.0950,73.91,3.21,13.20,12.17,0.1301,0.8500,0.3621,0.0722,1.31,0.09,1.4151,FDX,2018-11-30
PAYX,NaN,359091176,359091176,1,7002400000,5.290000e+09,4577900000,4.306800e+09,2424500000,0,0,938600000,0,8.589000e+08,235800000,235800000,0.66,0.65,0.56,2.235000e+08,1.203000e+08,-2.733000e+08,70500000,510600000,36600000,69.45,76.02,62.89,0.4361,0.1366,6.75,10.44,26.51,21.27,0.8085,0.0000,0.3462,0.2799,0.49,0.52,1.2283,PAYX,2018-11-30


In [91]:
df.isna().sum()

Quarter end                                   755
Shares                                          0
Shares split adjusted                           0
Split factor                                    0
Assets                                          0
Current Assets                                122
Liabilities                                     0
Current Liabilities                           122
Shareholders equity                             0
Non-controlling interest                        0
Preferred equity                                0
Goodwill & intangibles                          0
Long-term debt                                  0
Revenue                                         3
Earnings                                        0
Earnings available for common stockholders      0
EPS basic                                       0
EPS diluted                                     0
Dividend per share                              0
Cash from operating activities                  2


In [92]:
#removing the Quarter End column since the date for quarter_end has already been given
df.drop(columns=['Quarter end'], inplace=True)

In [93]:
#both current assets and current liabilities have 122 missing values each
df[(df['Current Assets'].isna()) & (df['Current Liabilities'].isna())]

,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker,quarter_end
ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DE,318570788,318570788,1,70108000000,NaN,58802800000,NaN,11287800000,17400000,0,4663100000,27237400000,9.415900e+09,784900000,784800000,2.46,2.39,0.690,2.495000e+09,-1.689400e+09,-7.796000e+08,-19300000,3904000000,584700000,145.45,159.30,131.60,0.2293,0.0343,35.43,4.52,22.76,27.53,0.3385,2.4130,0.1610,0.0634,0.54,6.00,NaN,DE,2018-10-28
JPM,3325410725,3325410725,1,2615183000000,NaN,2356227000000,NaN,258956000000,0,27764000000,0,0,2.726000e+10,8380000000,8380000000,2.35,2.34,0.800,1.318900e+10,-8.080000e+08,2.553000e+09,13917000000,419097000000,0,110.72,119.24,102.20,0.1226,0.0115,69.52,1.61,14.78,30.52,0.2961,0.0000,0.0884,0.2641,0.04,3.97,NaN,JPM,2018-09-30
DHI,388133243,388133243,1,14114600000,NaN,4955700000,NaN,8984400000,174500000,0,109200000,0,4.505300e+09,466200000,466200000,1.25,1.23,0.125,2.387000e+08,3.730000e+07,1.890000e+07,294900000,1473100000,-3600000,42.74,46.91,38.58,0.1735,0.1084,23.15,1.87,12.53,4.83,0.1300,0.0000,0.6365,0.0909,1.19,0.62,NaN,DHI,2018-09-30
AIV,155644246,155644246,1,6185021000,NaN,3889571000,NaN,2125682000,169768000,125000000,0,3646789000,2.424810e+08,569991000,567029000,3.62,3.61,0.380,1.173000e+08,5.328090e+08,-6.847500e+08,-34641000,104299000,-493405000,43.04,44.90,41.18,0.5558,0.1473,12.85,4.54,18.47,36.74,0.2577,1.8228,0.3235,0.9175,0.16,3.92,NaN,AIV,2018-09-30
MS,1720154771,1720154771,1,865517000000,NaN,785443000000,NaN,78703000000,1371000000,911000000,8920000000,189949000000,9.872000e+09,2112000000,2019000000,1.19,1.17,0.300,1.456500e+10,-4.763000e+09,1.536000e+09,10892000000,92481000000,453000000,48.90,51.52,46.28,0.0948,0.0091,45.22,1.09,12.41,28.55,0.2513,2.4418,0.0899,0.1786,0.05,8.20,NaN,MS,2018-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MER,1599731906,1599731906,1,875780000000,NaN,837425000000,NaN,38355000000,0,8605000000,4989000000,0,1.600000e+07,-5152000000,-7471000000,-5.58,-5.58,0.350,2.846100e+10,-7.850000e+08,-2.248100e+10,5195000000,36406000000,139000000,NaN,NaN,NaN,-0.9390,-0.0221,18.60,NaN,NaN,9.52,NaN,0.0000,0.0340,NaN,NaN,17.70,NaN,MER,2008-09-26
SAF,89944301,89944301,1,12348800000,NaN,8938100000,NaN,3384900000,25800000,0,0,0,1.518000e+09,149500000,149500000,1.65,1.65,0.400,2.002000e+08,9.090000e+07,-1.214000e+08,169700000,544300000,0,NaN,NaN,NaN,0.1827,0.0501,37.63,NaN,NaN,16.99,0.2311,0.0000,0.2741,0.1019,0.49,2.23,NaN,SAF,2008-06-30
LEH,694401926,694401926,1,639432000000,NaN,613156000000,NaN,26276000000,0,6993000000,0,0,NaN,-2774000000,-2873000000,-5.14,-5.14,0.170,-7.261000e+09,-2.570000e+08,6.467000e+09,-1051000000,6513000000,248000000,NaN,NaN,NaN,-0.0321,-0.0007,27.77,NaN,NaN,3.18,NaN,0.0000,0.0302,NaN,NaN,-10.81,NaN,LEH,2008-05-31


In [95]:
df['Current Assets'] = df['Current Assets'].fillna(0)
df['Current Liabilities'] = df['Current Liabilities'].fillna(0)

In [127]:
del df['ticker']

KeyError: ignored

In [128]:
temp2 = df.reset_index().groupby(['ticker'])['quarter_end'].nunique().reset_index()

In [129]:
temp2

,ticker,quarter_end
0,A,1
1,AA,1
2,AAN,1
3,AAP,1
4,AAPL,1
...,...,...
750,YHOO,1
751,YUM,1
752,ZION,1
753,ZMH,1


In [118]:
df['quarter_end'] = pd.to_datetime(df['quarter_end'])
df['Year'] = df['quarter_end'].dt.year
df['Quarter'] = df['Year'].astype(str) + 'Q' + df['quarter_end'].dt.quarter.astype(str)
df['Yearmonth'] = df['Year'].astype(str) + 'M' + df['quarter_end'].dt.month.astype(str)

In [123]:
df.Year.unique()

array([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008,
       2007, 2006])

In [137]:
df.groupby(['Year','Quarter'])['quarter_end'].count()

Year  Quarter
2006  2006Q3       1
2007  2007Q3       1
2008  2008Q1       4
      2008Q2       4
      2008Q3       9
      2008Q4       2
2009  2009Q1       1
      2009Q2       2
      2009Q3       3
      2009Q4       5
2010  2010Q1       1
      2010Q2       2
      2010Q3       3
      2010Q4       3
2011  2011Q1       4
      2011Q2       1
      2011Q4       4
2012  2012Q1       3
      2012Q2       1
      2012Q3       2
      2012Q4       1
2013  2013Q1       2
      2013Q2       1
      2013Q3       6
      2013Q4       2
2014  2014Q1       2
      2014Q2       1
      2014Q3       3
      2014Q4       7
2015  2015Q1       3
      2015Q2       7
      2015Q3       6
      2015Q4       3
2016  2016Q1       6
      2016Q2       7
      2016Q3       6
      2016Q4       5
2017  2017Q1       4
      2017Q2       5
      2017Q3       3
      2017Q4       4
2018  2018Q1       4
      2018Q2       7
      2018Q3     539
      2018Q4      65
Name: quarter_end, dtype: int64

In [147]:
df_2018 = df[(df['Year']== 2018) & (df['Quarter'] == '2018Q3')]

In [151]:
df_2018.isnull().sum()

Shares                                          0
Shares split adjusted                           0
Split factor                                    0
Assets                                          0
Current Assets                                  0
Liabilities                                     0
Current Liabilities                             0
Shareholders equity                             0
Non-controlling interest                        0
Preferred equity                                0
Goodwill & intangibles                          0
Long-term debt                                  0
Revenue                                         0
Earnings                                        0
Earnings available for common stockholders      0
EPS basic                                       0
EPS diluted                                     0
Dividend per share                              0
Cash from operating activities                  2
Cash from investing activities                  2


In [150]:
df.groupby(['Yearmonth', 'Quarter'])['Quarter'].count()

Yearmonth  Quarter
2006M9     2006Q3       1
2007M9     2007Q3       1
2008M12    2008Q4       2
2008M2     2008Q1       1
2008M3     2008Q1       3
                     ... 
2018M2     2018Q1       2
2018M3     2018Q1       2
2018M6     2018Q2       7
2018M8     2018Q3      16
2018M9     2018Q3     523
Name: Quarter, Length: 65, dtype: int64

In [100]:
df['quarter_end'].unique()

array(['2018-11-30', '2018-11-29', '2018-11-25', '2018-11-17',
       '2018-11-10', '2018-11-04', '2018-11-03', '2018-11-02',
       '2018-10-31', '2018-10-28', '2018-10-27', '2018-10-26',
       '2018-10-06', '2018-09-30', '2018-09-29', '2018-09-28',
       '2018-09-23', '2018-09-08', '2018-09-01', '2018-08-31',
       '2018-08-26', '2018-06-30', '2018-03-31', '2018-02-28',
       '2018-02-04', '2017-12-31', '2017-12-29', '2017-09-30',
       '2017-07-29', '2017-07-02', '2017-06-30', '2017-04-01',
       '2017-03-31', '2017-01-01', '2016-12-31', '2016-11-30',
       '2016-10-02', '2016-10-01', '2016-09-30', '2016-09-23',
       '2016-07-03', '2016-06-30', '2016-06-24', '2016-04-03',
       '2016-04-01', '2016-03-31', '2015-12-31', '2015-09-30',
       '2015-09-27', '2015-09-25', '2015-06-30', '2015-05-30',
       '2015-04-30', '2015-03-31', '2015-03-28', '2014-12-31',
       '2014-12-26', '2014-11-30', '2014-11-02', '2014-11-01',
       '2014-09-30', '2014-09-06', '2014-06-28', '2014-

In [130]:
df.head(3)

,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,quarter_end,Year,Quarter,Yearmonth
ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ORCL,3588919000,3588919000,1,118318000000,5.693800e+10,87263000000,2.031500e+10,30572000000,483000000,0,49881000000,51561000000,9.562000e+09,2333000000,2333000000,0.63,0.61,0.19,5.460000e+08,1.920000e+09,-1.002300e+10,-7631000000,10824000000,421000000,49.23,52.11,46.34,0.0982,0.0305,8.52,4.90,54.10,4.37,0.7371,1.6865,0.2584,0.1001,0.30,0.03,2.8028,2018-11-30,2018,2018Q4,2018M11
FDX,261043625,261043625,1,53289000000,1.343800e+10,33995000000,9.496000e+09,19294000000,0,0,6908000000,16399000000,1.782400e+10,935000000,933000000,3.56,3.51,0.65,1.478000e+09,-1.480000e+09,-2.300000e+08,-246000000,2123000000,1455000000,233.57,259.25,207.90,0.2586,0.0950,73.91,3.21,13.20,12.17,0.1301,0.8500,0.3621,0.0722,1.31,0.09,1.4151,2018-11-30,2018,2018Q4,2018M11
PAYX,359091176,359091176,1,7002400000,5.290000e+09,4577900000,4.306800e+09,2424500000,0,0,938600000,0,8.589000e+08,235800000,235800000,0.66,0.65,0.56,2.235000e+08,1.203000e+08,-2.733000e+08,70500000,510600000,36600000,69.45,76.02,62.89,0.4361,0.1366,6.75,10.44,26.51,21.27,0.8085,0.0000,0.3462,0.2799,0.49,0.52,1.2283,2018-11-30,2018,2018Q4,2018M11


In [131]:
#Shareholders equity = Asses - Liabilities - Non-controlling interest
#Reviewing that this equation holds true in this dataset 
temp = df.copy()
temp['shareholders_equity'] = temp['Assets'] - temp['Liabilities'] - temp['Non-controlling interest']
temp['diff'] = temp['shareholders_equity'] - temp['Shareholders equity']

In [132]:
temp[np.abs(temp['diff'] != 0)]

,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,quarter_end,Year,Quarter,Yearmonth,shareholders_equity,diff
ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [38]:
df[df.Revenue.isnull()]

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker,quarter_end
735,NaN,666161708,666161708,1,77093668000,NaN,71348993000,NaN,5596714000,147961000,195445000,3431481000,0,NaN,-1603193000,-1603193000,-2.43,-2.43,0.00,4.639720e+08,-1.436690e+08,9.085240e+08,1228827000,3754523000,15599000,3.73,5.84,1.63,-0.3489,-0.0299,8.11,0.35,NaN,1.66,NaN,0.0,0.0701,NaN,NaN,0.67,NaN,SOV,2008-12-31
740,NaN,2160916994,2160916994,1,764378000000,NaN,711392000000,NaN,50003000000,2983000000,0,20211000000,0,NaN,-23698000000,-23889000000,-11.18,-11.18,0.05,6.166000e+09,-1.176000e+09,-1.790900e+10,-12919000000,34420000000,602000000,NaN,NaN,NaN,-0.4808,-0.0419,23.14,NaN,NaN,21.96,NaN,0.0,0.0654,NaN,NaN,2.57,NaN,WB,2008-09-30
748,NaN,694401926,694401926,1,639432000000,NaN,613156000000,NaN,26276000000,0,6993000000,0,0,NaN,-2774000000,-2873000000,-5.14,-5.14,0.17,-7.261000e+09,-2.570000e+08,6.467000e+09,-1051000000,6513000000,248000000,NaN,NaN,NaN,-0.0321,-0.0007,27.77,NaN,NaN,3.18,NaN,0.0,0.0302,NaN,NaN,-10.81,NaN,LEH,2008-05-31


In [43]:
temp['book_value_calculated'] = (temp['Shareholders equity'] - temp['Preferred equity'])
temp['book_value_per_share_calculated'] = temp['book_value_calculated']/temp['Shares']

In [47]:
temp.head()

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker,quarter_end,shareholders_equity,diff,book_value_calculated,book_value_per_share_calculated
0,NaN,3588919000,3588919000,1,118318000000,5.693800e+10,87263000000,2.031500e+10,30572000000,483000000,0,49881000000,51561000000,9.562000e+09,2333000000,2333000000,0.63,0.61,0.19,5.460000e+08,1.920000e+09,-1.002300e+10,-7631000000,10824000000,421000000,49.23,52.11,46.34,0.0982,0.0305,8.52,4.90,54.10,4.37,0.7371,1.6865,0.2584,0.1001,0.30,0.03,2.8028,ORCL,2018-11-30,30572000000,0,30572000000,8.518442
1,NaN,261043625,261043625,1,53289000000,1.343800e+10,33995000000,9.496000e+09,19294000000,0,0,6908000000,16399000000,1.782400e+10,935000000,933000000,3.56,3.51,0.65,1.478000e+09,-1.480000e+09,-2.300000e+08,-246000000,2123000000,1455000000,233.57,259.25,207.90,0.2586,0.0950,73.91,3.21,13.20,12.17,0.1301,0.8500,0.3621,0.0722,1.31,0.09,1.4151,FDX,2018-11-30,19294000000,0,19294000000,73.911018
2,NaN,359091176,359091176,1,7002400000,5.290000e+09,4577900000,4.306800e+09,2424500000,0,0,938600000,0,8.589000e+08,235800000,235800000,0.66,0.65,0.56,2.235000e+08,1.203000e+08,-2.733000e+08,70500000,510600000,36600000,69.45,76.02,62.89,0.4361,0.1366,6.75,10.44,26.51,21.27,0.8085,0.0000,0.3462,0.2799,0.49,0.52,1.2283,PAYX,2018-11-30,2424500000,0,2424500000,6.751767
3,NaN,665541059,665541059,1,26706412000,1.354041e+10,13650844000,1.017634e+10,12678856000,376712000,0,6219615000,19896000,1.060555e+10,1274720000,1274720000,2.00,1.96,1.46,1.027508e+09,-2.728680e+08,-1.462168e+09,-697570000,4363790000,72892000,163.13,175.64,150.63,0.3961,0.1727,19.05,10.44,25.73,17.64,0.4436,0.0016,0.4747,0.0999,1.73,1.43,1.3306,ACN,2018-11-30,12678856000,0,12678856000,19.050449
4,NaN,943444736,943444736,1,69941000000,2.008300e+10,43678000000,2.509900e+10,25609000000,654000000,0,28393000000,11646000000,3.379300e+10,1123000000,1123000000,1.18,1.18,0.44,4.590000e+08,-6.350000e+08,3.900000e+08,208000000,1183000000,470000000,76.37,85.31,67.42,0.1996,0.0763,27.14,2.79,15.09,5.74,0.3054,0.4548,0.3662,0.0396,1.93,-0.01,0.8002,WBA,2018-11-30,25609000000,0,25609000000,27.144144


In [48]:
temp['calculated_stock_price'] = temp['P/B ratio']*temp['Book value of equity per share']

temp.loc[temp.Price.isnull(), 'Price'] = temp['calculated_stock_price']
temp.loc[temp['Price high'].isnull(), 'Price high'] = temp['calculated_stock_price']
temp.loc[temp['Price low'].isnull(), 'Price low'] = temp['calculated_stock_price']


In [57]:
temp['calculated_net_income'] = temp['Net margin']*temp['Revenue']
temp['calculated_roe'] = temp['calculated_net_income']/temp['Shareholders equity']

temp['calculated_pe_ratio'] = temp['Price']/temp['EPS basic']

In [58]:
temp.head()

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker,quarter_end,shareholders_equity,diff,book_value_calculated,book_value_per_share_calculated,calculated_stock_price,calculated_roe,calculated_net_income,calculated_pe_ratio
0,NaN,3588919000,3588919000,1,118318000000,5.693800e+10,87263000000,2.031500e+10,30572000000,483000000,0,49881000000,51561000000,9.562000e+09,2333000000,2333000000,0.63,0.61,0.19,5.460000e+08,1.920000e+09,-1.002300e+10,-7631000000,10824000000,421000000,49.23,52.11,46.34,0.0982,0.0305,8.52,4.90,54.10,4.37,0.7371,1.6865,0.2584,0.1001,0.30,0.03,2.8028,ORCL,2018-11-30,30572000000,0,30572000000,8.518442,41.7480,0.031308,9.571562e+08,78.142857
1,NaN,261043625,261043625,1,53289000000,1.343800e+10,33995000000,9.496000e+09,19294000000,0,0,6908000000,16399000000,1.782400e+10,935000000,933000000,3.56,3.51,0.65,1.478000e+09,-1.480000e+09,-2.300000e+08,-246000000,2123000000,1455000000,233.57,259.25,207.90,0.2586,0.0950,73.91,3.21,13.20,12.17,0.1301,0.8500,0.3621,0.0722,1.31,0.09,1.4151,FDX,2018-11-30,19294000000,0,19294000000,73.911018,237.2511,0.066699,1.286893e+09,65.609551
2,NaN,359091176,359091176,1,7002400000,5.290000e+09,4577900000,4.306800e+09,2424500000,0,0,938600000,0,8.589000e+08,235800000,235800000,0.66,0.65,0.56,2.235000e+08,1.203000e+08,-2.733000e+08,70500000,510600000,36600000,69.45,76.02,62.89,0.4361,0.1366,6.75,10.44,26.51,21.27,0.8085,0.0000,0.3462,0.2799,0.49,0.52,1.2283,PAYX,2018-11-30,2424500000,0,2424500000,6.751767,70.4700,0.099157,2.404061e+08,105.227273
3,NaN,665541059,665541059,1,26706412000,1.354041e+10,13650844000,1.017634e+10,12678856000,376712000,0,6219615000,19896000,1.060555e+10,1274720000,1274720000,2.00,1.96,1.46,1.027508e+09,-2.728680e+08,-1.462168e+09,-697570000,4363790000,72892000,163.13,175.64,150.63,0.3961,0.1727,19.05,10.44,25.73,17.64,0.4436,0.0016,0.4747,0.0999,1.73,1.43,1.3306,ACN,2018-11-30,12678856000,0,12678856000,19.050449,198.8820,0.083564,1.059494e+09,81.565000
4,NaN,943444736,943444736,1,69941000000,2.008300e+10,43678000000,2.509900e+10,25609000000,654000000,0,28393000000,11646000000,3.379300e+10,1123000000,1123000000,1.18,1.18,0.44,4.590000e+08,-6.350000e+08,3.900000e+08,208000000,1183000000,470000000,76.37,85.31,67.42,0.1996,0.0763,27.14,2.79,15.09,5.74,0.3054,0.4548,0.3662,0.0396,1.93,-0.01,0.8002,WBA,2018-11-30,25609000000,0,25609000000,27.144144,75.7206,0.052255,1.338203e+09,64.720339
